# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

In [16]:
#include <stdio.h>

int main(){
    int n;
    float s, j, p;
    s = 200.0;
    n = 5;
    j = 0.01;
    p = s;
    printf("Parcela 1: R$%.2f.\n", p);
    for (int i = 2; i <= n; i++){
        p = p *(1 + j);
        printf("Parcela %d: R$%.2f.\n", i, p);
    }
    return 0;
}

Parcela 1: R$200.00.
Parcela 2: R$202.00.
Parcela 3: R$204.02.
Parcela 4: R$206.06.
Parcela 5: R$208.12.


## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de uma única parcela X do empréstimo. Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa principal. A função deve apenas calcular uma única parcela, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [15]:
#include <stdio.h>

float proximaParcela(float s, float j, int x){
    float p;
    if (x == 1){
        p = s;
    }
    else{
        p = proximaParcela(s, j, x - 1) * (1 + j);
    }
    return p;
}

int main(){
    int n;
    float s, j, p;
    s = 200.0;
    n = 5;
    j = 0.01;
    p = s;
    for (int i = 1; i <= n; i++){
        printf("Parcela %d: R$%.2f.\n", i, proximaParcela(p, j, i));
    }
    return 0;
}

Parcela 1: R$200.00.
Parcela 2: R$202.00.
Parcela 3: R$204.02.
Parcela 4: R$206.06.
Parcela 5: R$208.12.


## Exercício Parte 3 - Mantendo o Estado

Modifique a função (módulo) `proximaParcela` acima de modo que ela tenha uma memória da situação em que estão as coisas, sem que o `main` precise ficar informando novamente coisas que já informou no início. Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência dos módulos com o programa principal.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente.

In [14]:
#include <stdio.h>

/* np é o número da parcela atual. */
int n, np;
float j, s, p;

void proximaParcela(){
    if (np < n){
        p = p * (1 + j);
        np++;
    }
    else{
        p = 0;
    }
}

int main(){
    s = 200.0;
    n = 5;
    j = 0.01;
    np = 1;
    p = s;
    while (p > 0){
        printf("Parcela %d: R$%.2f.\n", np, p);
        proximaParcela();
    }
    return 0;
}

Parcela 1: R$200.00.
Parcela 2: R$202.00.
Parcela 3: R$204.02.
Parcela 4: R$206.06.
Parcela 5: R$208.12.


## Exercício Parte 4 - Minimizando os Parâmetros

Retomando a solução da `Parte 2`, em que a função `main` deve ter usado vários parâmetros para a comunicação com a função `proximaParcela`,  modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar o estado completo do empréstimo.

In [13]:
#include <stdio.h>
#include <stdlib.h>

typedef struct Emprestimo{
    int n, np; /* np é o número da parcela atual. */
    float s, p, j;
} Emprestimo;

Emprestimo * inicializar_emprestimo(float s, int n, float j){
    Emprestimo *emprestimo;
    emprestimo = malloc(sizeof(Emprestimo));
    emprestimo->n = n;
    emprestimo->np = 1;
    emprestimo->s = s;
    emprestimo->p = s;
    emprestimo->j = j;
    return emprestimo;
}

void proximaParcela(Emprestimo *emprestimo){
    if (emprestimo->np < emprestimo->n){
        emprestimo->p = emprestimo->p * (1 + emprestimo->j);
        emprestimo->np++;
    }
    else{
        emprestimo->p = 0;
    }
}

int main(){
    Emprestimo *emprestimo;
    emprestimo = inicializar_emprestimo(200.0, 5, 0.01);
    while (emprestimo->p > 0){
        printf("Parcela %d: R$%.2f.\n", emprestimo->np, emprestimo->p);
        proximaParcela(emprestimo);
    }
    return 0;
}

Parcela 1: R$200.00.
Parcela 2: R$202.00.
Parcela 3: R$204.02.
Parcela 4: R$206.06.
Parcela 5: R$208.12.


## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada empréstimo separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, a função `proximaParcela` ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que, cada novo empréstimo só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo.

In [12]:
#include <stdio.h>
#include <stdlib.h>

typedef struct Emprestimo{
    int n, np; /* np é o número da parcela atual. */
    float s, p, j;
} Emprestimo;

Emprestimo * inicializar_emprestimo(float s, int n, float j){
    Emprestimo *emprestimo;
    emprestimo = malloc(sizeof(Emprestimo));
    emprestimo->n = n;
    emprestimo->np = 1;
    emprestimo->s = s;
    emprestimo->p = s;
    emprestimo->j = j;
    return emprestimo;
}

void proximaParcela(Emprestimo *emprestimo){
    if (emprestimo->np < emprestimo->n){
        emprestimo->p = emprestimo->p * (1 + emprestimo->j);
        emprestimo->np++;
    }
    else{
        emprestimo->p = 0;
    }
}

int main(){
    int fim, tamanho;
    Emprestimo **emprestimos;
    tamanho = 2;
    emprestimos = malloc(tamanho * sizeof(Emprestimo *));
    emprestimos[0] = inicializar_emprestimo(200.0, 5, 0.01);
    emprestimos[1] = inicializar_emprestimo(500.0, 7, 0.02);
    fim = 0;
    while (!fim){
        fim = 1;
        for (int i = 1; i < 3; i++){
            if (emprestimos[i - 1]->p > 0){
                fim = 0;
                printf("Empréstimo %d | Parcela %d: R$%.2f.\n", i, emprestimos[i - 1]->np, emprestimos[i - 1]->p);
                proximaParcela(emprestimos[i - 1]);
            }
        }
    }
    return 0;
}

Empréstimo 1 | Parcela 1: R$200.00.
Empréstimo 2 | Parcela 1: R$500.00.
Empréstimo 1 | Parcela 2: R$202.00.
Empréstimo 2 | Parcela 2: R$510.00.
Empréstimo 1 | Parcela 3: R$204.02.
Empréstimo 2 | Parcela 3: R$520.20.
Empréstimo 1 | Parcela 4: R$206.06.
Empréstimo 2 | Parcela 4: R$530.60.
Empréstimo 1 | Parcela 5: R$208.12.
Empréstimo 2 | Parcela 5: R$541.22.
Empréstimo 2 | Parcela 6: R$552.04.
Empréstimo 2 | Parcela 7: R$563.08.
